In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain.prompts import PromptTemplate

In [3]:
from dotenv import load_dotenv  
load_dotenv()  

True

In [4]:
import warnings
warnings.filterwarnings("ignore") 

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.5
)

### Text Classification

#### Prompt Templates

In [8]:
prompt_templates_text_classification  = {
    "Zero-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Categorize the following headline.\n"
            "Sentence: {sentence}\n"
            "Classification:"
        )
    ),
    "One-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Categorize based on the following examples:\n"
            "Example: 'Apple unveils new iPhone' → technology\n"
            "Now, categorize the following headline: {sentence}\n"
            "Category:"
        )
    ),
    "Few-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Categorize based on the following examples:\n"
            "Text: 'Healthy eating is important.' → Category: Health\n"
            "Text: 'Cryptocurrency market is on the rise.' → Category: Economy\n"
            "Text: 'The US economy is growing.' → Category: Economy\n"
            "Text: 'What a lovely day' → Positive\n"
            "Now, categorize the following headline: {sentence}\n"
            "Category:"
        )
    ),
    "Context-learning": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "You are a classification assistant.\n"
            "Please respond the following sentence.\n" 
            "Sentence: {sentence}\n"
            "Category:"
        )
    ),
    "Chain-of-Thought": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Categorize the following headline.\n"
            "Think step by step before giving the final category, then decide which category fits best.\n"
            "Sentence: {sentence}\n"
            "Category:"
        )
    )
}

#### Testing Prompts Function

In [15]:
def test_prompts(sentence: str, prompt_template):
    print(f"\n Input sentence: {sentence}\n{'-'*60}")
    for name, prompt in prompt_template.items():
        prompt_message = prompt.format(sentence=sentence)
        response = llm.invoke([HumanMessage(content=prompt_message)])
        print(f"\n✅ {name} → {response.content}")

#### Testing

In [18]:
user = "The ongoing debate among theoretical physicists regarding the implications of M-theory for a unified 'theory of everything' not only pushes the boundaries of cosmology and quantum mechanics but also increasingly intersects with philosophical inquiries into the nature of reality and the limits of human perception."
# Physics / Philosophy
test_prompts(user, prompt_templates_text_classification)


 Input sentence: The ongoing debate among theoretical physicists regarding the implications of M-theory for a unified 'theory of everything' not only pushes the boundaries of cosmology and quantum mechanics but also increasingly intersects with philosophical inquiries into the nature of reality and the limits of human perception.
------------------------------------------------------------

✅ Zero-shot → **Theoretical Physics/Science & Philosophy**

**Explanation:**

The headline clearly deals with:

*   **Theoretical Physics:** Mention of M-theory, cosmology, and quantum mechanics places it firmly within this field.
*   **Science & Philosophy:** The explicit mention of philosophical inquiries into the nature of reality and the limits of human perception connects it to philosophical discussions.

Therefore, a combined classification best reflects the content.

✅ One-shot → Physics/Science

✅ Few-shot → Physics/Science/Philosophy (This could fit into multiple categories, but these are 

In [24]:
user = "Despite the film's stunning cinematography and A-list cast, its convoluted plot and excessive runtime left audiences polarized, sparking heated debates among critics."
# Movie Review
test_prompts(user, prompt_templates_text_classification)


 Input sentence: Despite the film's stunning cinematography and A-list cast, its convoluted plot and excessive runtime left audiences polarized, sparking heated debates among critics.
------------------------------------------------------------

✅ Zero-shot → **Film Review/Entertainment News**

**Explanation:**

The headline discusses aspects of a film (cinematography, cast, plot, runtime), its reception by audiences (polarized), and critical response (heated debates). These are all typical elements covered in film reviews and entertainment news reporting.

✅ One-shot → entertainment

✅ Few-shot → Movies

✅ Context-learning → Movie Review

✅ Chain-of-Thought → Let's analyze the headline step by step:

1.  **Keywords:** "film," "cinematography," "A-list cast," "plot," "runtime," "audiences polarized," "critics," "debates."
2.  **Content:** The headline discusses the reception of a film, highlighting both positive and negative aspects. It mentions audience and critic reactions.
3.  **Po

Most effective prompts are Context-learning and Chain-of-Thought for this task.

### Information Extraction

#### Prompts

In [26]:
prompt_templates_information_extraction  = {
    "Zero-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Extract information from the sentence:\n"
            "Sentence: {sentence}\n"
        )
    ),
    "One-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Example:\n"
            "Elon Musk, CEO of SpaceX, spoke about Mars colonization at the Future Tech Summit in San Francisco on May 5, 2023.\n"
            "- Person: Elon Musk\n"
            "- Organization: SpaceX\n"
            "- Topic: Mars colonization\n"
            "- Event: Future Tech Summit\n"
            "- Location: San Francisco\n"
            "- Date: May 5, 2023\n"

            "Now extract the same information from the following sentence: {sentence}\n"
        )
    ),
    "Few-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Example 1:"
            "Barack Obama visited Berlin on June 19, 2013.\n"
            "- Person: Barack Obama\n"
            "- Location: Berlin\n"
            "- Date: June 19, 2013\n"

            "Example 2:"
            "Noam Chomsky laid the foundations of natural language processing (NLP) by developing the generative grammar model to understand the structure of language.\n"
            "- Person: Noam Chomsky\n"
            "- Action: Developed generative grammar model\n"
            "- Purpose: To understand language structure\n"
            "- Impact: Laid NLP foundations\n"
            "- Domain: Computational linguistics\n"
            "Now extract information from this sentence: {sentence}"
        )
    ),
    "Context-learning": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "You are an information extraction system. Identify and list all entities from the given sentence:\n"
            "Sentence: {sentence}\n"
        )
    ),
    "Chain-of-Thought": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Think step by step. Then identify all the entities in the sentence for information extraction.\n"
            "Sentence: {sentence}\n"
        )
    )
}

#### Testing

In [29]:
user = "Dr. Emily Carter from Princeton University presented her research on quantum materials at the International Physics Conference in Geneva on July 12, 2024."

test_prompts(user, prompt_templates_information_extraction)


 Input sentence: Dr. Emily Carter from Princeton University presented her research on quantum materials at the International Physics Conference in Geneva on July 12, 2024.
------------------------------------------------------------

✅ Zero-shot → Here's the extracted information from the sentence:

*   **Who:** Dr. Emily Carter
*   **Affiliation:** Princeton University
*   **What:** Presented research on quantum materials
*   **Where:** International Physics Conference in Geneva
*   **When:** July 12, 2024

✅ One-shot → Here's the extracted information:

- Person: Dr. Emily Carter
- Organization: Princeton University
- Topic: Quantum materials
- Event: International Physics Conference
- Location: Geneva
- Date: July 12, 2024

✅ Few-shot → Here's the extracted information from the sentence:

- Person: Dr. Emily Carter
- Affiliation: Princeton University
- Action: Presented research
- Topic: Quantum materials
- Event: International Physics Conference
- Location: Geneva
- Date: July 12,

In [32]:
user = "Despite initial skepticism from the AI community, Dr. Liang's 2021 paper on neurosymbolic hybrid models published in Nature ML after a controversial peer-review—was later validated by Google DeepMind’s experiments in early 2023, achieving a 17% accuracy boost on complex reasoning tasks."

test_prompts(user, prompt_templates_information_extraction)


 Input sentence: Despite initial skepticism from the AI community, Dr. Liang's 2021 paper on neurosymbolic hybrid models published in Nature ML after a controversial peer-review—was later validated by Google DeepMind’s experiments in early 2023, achieving a 17% accuracy boost on complex reasoning tasks.
------------------------------------------------------------

✅ Zero-shot → Here's the information extracted from the sentence:

*   **Subject of the sentence:** Dr. Liang's 2021 paper on neurosymbolic hybrid models
*   **Publication Venue:** Nature ML
*   **Publication Year:** 2021
*   **Validation:** Validated by Google DeepMind’s experiments
*   **Validation Year:** Early 2023
*   **Outcome of Validation:** Achieved a 17% accuracy boost on complex reasoning tasks
*   **Contextual Information:**
    *   Initial skepticism from the AI community
    *   Controversial peer-review process

✅ One-shot → Here's the extracted information from the sentence:

*   **Person:** Dr. Liang
*   **O

Most effective prompts are Few-shot and Chain-of-Thought for this task.

### Creative Writing

#### Prompts

In [33]:
prompt_templates_creative_writing   = {
    "Zero-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Write creative writing based on the following sentence\n"
            "Sentence: {sentence}\n"
        )
    ),
    "One-shot": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Example:"
            "Once upon a time, in a city where dreams were stored in glass bottles, a young girl set out to steal them back.\n"
            "→ A surreal and emotional opening line for a fantasy short story.\n"

            "Now, using this as inspiration, craft an equally imaginative write for your own piece of writing based on the sentence I will provide.\n"
            "Sentence: {sentence}\n"
            
        )
    ),
    "Multi‑perspective": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Based on the following sentence, write a piece of creative writing (story, dialogue, poem, etc.) that includes at least two different perspectives or voices.\n"
            "Make sure each perspective has its own tone and personality, and let them interact in a way that feels natural and thought-provoking.\n"
            "Sentence: {sentence}\n"

        )
    ),
    "Context-learning": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "You are an imaginative novelist.\n"
            "Write creative writing based on the following sentence\n"
            "Sentence: {sentence}\n"
        )
    ),
    "Chain-of-Thought": PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Before writing, brainstorm key elements step by step (characters, setting, tone, conflict)." 
            "After listing them, use those elements to produce a piece of creative writing that fulfills the sentence I give you."
            "Sentence: {sentence}\n"
        )
    )
}

#### Testing

In [36]:
user = "A conversation between the moon and the ocean."

test_prompts(user, prompt_templates_creative_writing)


 Input sentence: A conversation between the moon and the ocean.
------------------------------------------------------------

✅ Zero-shot → The moon, a silver coin tossed carelessly into the velvet sky, sighed. "Another night, another tide. Tell me, Ocean, are you ever weary of this endless ebb and flow?"

The ocean, a vast, restless creature of inky depths and foamy crests, chuckled, a sound like pebbles grinding against each other. "Weary? My dear Luna, I am the breath of the world. To be still is to be stagnant, to be stagnant is to be dead. You, on the other hand, seem rather… static."

"Static?" The moon's light flickered, a momentary eclipse caused by a passing cloud. "I guide you, Ocean. I pull you from your slumber, I dictate your rhythms. Surely that is not static."

"Guide, perhaps," the ocean conceded, its waves licking greedily at the shore. "But you do not feel. You do not taste the salt, the sting of the storm, the joy of a newborn coral reef. You are a silent observer, 

In [38]:
user = "Magic door"

test_prompts(user, prompt_templates_creative_writing)


 Input sentence: Magic door
------------------------------------------------------------

✅ Zero-shot → The chipped paint on the door was the color of a bruised plum, a shade that screamed neglect and whispered secrets. It stood alone, nestled between the gnarled, ancient branches of an oak tree that seemed to hold its breath. No path led to it, no house surrounded it. Just the door, hanging on rusted hinges, a silent sentinel in the heart of the Whispering Woods.

It wasn't the door itself that drew you in, though. It was the feeling. A hum, a vibration that resonated deep in your bones, a pull that tugged at the very fabric of your being. It promised adventure, escape, a glimpse beyond the veil of the mundane. It was a magic door.

Elara had felt the pull for months, ever since she'd stumbled upon the clearing while fleeing the relentless drizzle. She'd dismissed it as fancy, a trick of the light, the product of too many fairy tales read in her youth. But the feeling persisted, grow

Most effective prompts are Context-learning and Chain-of-Thought for this task.

### Solving a mathematical or logical problem with Chain-of-Thought prompting

In [42]:
prompt= PromptTemplate(
        input_variables=["sentence"],
        template=(
            "Let's solve step by step carefully."
            "Problem: {sentence}"
            "Think step by step and explain your reasoning before giving the final answer."
        )
    )
problem = "A train travels 60 km in 1 hour. How long will it take to travel 300 km if its speed doubles every hour?"
prompt_message = prompt.format(sentence=problem)

In [44]:
responce = llm.invoke([HumanMessage(content=prompt_message)])
print(responce.content)

Okay, let's break this problem down step-by-step.

**Step 1: Determine the initial speed.**

The train travels 60 km in 1 hour, so its initial speed is 60 km/hour.

**Step 2: Track the speed and distance covered in each hour.**

*   **Hour 1:**
    *   Speed: 60 km/hour
    *   Distance covered: 60 km
    *   Total distance traveled: 60 km
*   **Hour 2:**
    *   Speed: 60 km/hour * 2 = 120 km/hour
    *   Distance covered: 120 km
    *   Total distance traveled: 60 km + 120 km = 180 km
*   **Hour 3:**
    *   Speed: 120 km/hour * 2 = 240 km/hour
    *   Distance covered: 240 km
    *   Total distance traveled: 180 km + 240 km = 420 km

**Step 3: Determine when the train reaches 300 km.**

After 2 hours, the train has traveled 180 km. After 3 hours, the train has traveled 420 km. This means the train reaches 300 km sometime between hour 2 and hour 3.

**Step 4: Calculate the time in the third hour to reach 300 km.**

At the end of hour 2, the train has traveled 180 km. It needs to trav

### Effective prompt templates

In [46]:
best_prompt_library = {
    "text_classification": {
        "best_context_learning": prompt_templates_text_classification["Context-learning"],
        "best_chain_of_thought": prompt_templates_text_classification["Chain-of-Thought"]
    },
    "information_extraction": {
        "best_few_shot": prompt_templates_information_extraction["Few-shot"],
        "best_chain_of_thought": prompt_templates_information_extraction["Chain-of-Thought"]
    },
    "creative_writing": {
        "best_chain_of_thought": prompt_templates_creative_writing["Chain-of-Thought"],
        "best_context_learning": prompt_templates_creative_writing["Context-learning"]
    }
}

In [49]:
chosen_prompt = best_prompt_library["text_classification"]["best_chain_of_thought"]

print(chosen_prompt.format(sentence="The stock market is booming today."))

Categorize the following headline.
Think step by step before giving the final category, then decide which category fits best.
Sentence: The stock market is booming today.
Category:
